In [7]:
import pandas as pd
import numpy as np

In [102]:
#1
data_all=pd.read_csv('Medicare_Data/2012MedicareProviderPayment.txt', delimiter='\t')

In [103]:
#fct rename columns lower letters; remove spaces
def lower_columns(pdx):
    cols = pdx.columns.tolist()
    cols = [col.lower() for col in cols]
    pdx.columns = cols
    pdx.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
    return pdx
data_all=lower_columns(data_all)

In [104]:

#cleaup first row
data_all= data_all[data_all.npi != 1]
#add report year
data_all['report_year']='2012'

#add zip code
data_all['nppes_provider_zip']=data_all['nppes_provider_zip'].astype(str)
data_all['nppes_provider_zip_base']=data_all['nppes_provider_zip'].str.slice(0,5)
#data_all['nppes_provider_zip_base'].unique()

In [105]:
data_all.columns

Index([u'npi', u'nppes_provider_last_org_name', u'nppes_provider_first_name',
       u'nppes_provider_mi', u'nppes_credentials', u'nppes_provider_gender',
       u'nppes_entity_code', u'nppes_provider_street1',
       u'nppes_provider_street2', u'nppes_provider_city',
       u'nppes_provider_zip', u'nppes_provider_state',
       u'nppes_provider_country', u'provider_type',
       u'medicare_participation_indicator', u'place_of_service', u'hcpcs_code',
       u'hcpcs_description', u'hcpcs_drug_indicator', u'line_srvc_cnt',
       u'bene_unique_cnt', u'bene_day_srvc_cnt',
       u'average_medicare_allowed_amt', u'stdev_medicare_allowed_amt',
       u'average_submitted_chrg_amt', u'stdev_submitted_chrg_amt',
       u'average_medicare_payment_amt', u'stdev_medicare_payment_amt',
       u'report_year', u'nppes_provider_zip_base'],
      dtype='object')

In [106]:
#select WA state providers
data_all['nppes_provider_state']=data_all['nppes_provider_state'].astype(str)
data_provider_wa=data_all[data_all['nppes_provider_state']=='WA']

In [107]:
#exploratory
data_provider_wa.shape
#data_provider_wa.info()

(174582, 28)

In [109]:
#remove all non-individual provider from data
data_provider_wa=data_provider_wa[data_provider_wa.nppes_provider_first_name.notnull()]

In [110]:
#remove all non-medicare providers
data_provider_wa=data_provider_wa[data_provider_wa['medicare_participation_indicator']=='Y']

In [111]:
#function to drop columns
drop_col=['nppes_entity_code','medicare_participation_indicator']
def drop_columns(col_remove,pdx):
    columns=list(pdx.columns.values)
    for col in col_remove:
        if col in columns:
            pdx.drop(col, axis=1, inplace=True)
    return pdx
#drop_columns(drop_col,data_provider_wa)       

In [112]:
# data_provider_wa_2014['test_x']='100'
# data_provider_wa_2014['test_y']='100'
# drop_col=['test_x','test_y']

In [113]:
data_provider_wa=drop_columns(drop_col,data_provider_wa)   

In [114]:
#data2=data1[data1.nppes_provider_first_name.isnull()]
#data_provider_wa_2014['participation_indicator'].unique()
#data_provider_wa_2014['bene_day_srvc_cnt'].value_counts()

In [115]:
#fct remane columns
def name_columns(value_old,value_new,pdx):
    cols = pdx.columns.tolist()
    cols = [col.replace(value_old,value_new) for col in cols]
    pdx.columns = cols
    return pdx



In [116]:
data_provider_wa=name_columns("nppes_","", data_provider_wa)
data_provider_wa=name_columns("provider_","", data_provider_wa)
data_provider_wa=name_columns("medicare_","", data_provider_wa)
#data_provider_wa_2014=name_columns("medicare_","", data_provider_wa_2014)
data_provider_wa=name_columns("average_","", data_provider_wa)

#replace NaN with '' for all columns
data_provider_wa.fillna('',inplace=True)


In [118]:
#fct remane columns
# def nan_replace(value,pdx):
#     cols = pdx.columns.tolist()
#     cols = [col.replace(value_old,value_new) for col in cols]
#     pdx.columns = cols
#     return pdx
# # str_cols = ['Name','City']
# str_cols = df.columns[df.dtypes==object]
# df[str_cols] = df[str_cols].fillna('.')

In [101]:
data_2013=data_provider_wa.copy()
data_2013.shape

(174582, 28)

In [119]:
data_2012=data_provider_wa.copy()
data_2012.shape

(172517, 28)

In [83]:
data_2014=data_provider_wa.copy()
data_2014.shape

(177510, 26)

In [151]:
data_2012.columns.tolist()

['npi',
 'last_org_name',
 'first_name',
 'mi',
 'credentials',
 'gender',
 'street1',
 'street2',
 'city',
 'zip',
 'state',
 'country',
 'type',
 'place_of_service',
 'hcpcs_code',
 'hcpcs_description',
 'hcpcs_drug_indicator',
 'line_srvc_cnt',
 'bene_unique_cnt',
 'bene_day_srvc_cnt',
 'allowed_amt',
 'stdev_allowed_amt',
 'submitted_chrg_amt',
 'stdev_submitted_chrg_amt',
 'payment_amt',
 'stdev_payment_amt',
 'report_year',
 'zip_base']

In [183]:
#remove all columns that are not common in all files


#set columns as sets to compare them
# these can be put in a function
set_col_2012=set(data_2012.columns)
set_col_2013=set(data_2013.columns)
set_col_2014=set(data_2014.columns)
#print set_col_2014
results_intersection = set.intersection(set_col_2012,set_col_2013,set_col_2014)

#print results_intersection
diff_2012=set_col_2012-results_intersection
diff_2013=set_col_2013-results_intersection
diff_2014=set_col_2014-results_intersection

#print diff_2014

results_union = list(set().union(diff_2012,diff_2013,diff_2014))

# #print results_union

data_2012=drop_columns(results_union,data_2012)  
data_2013=drop_columns(results_union,data_2013)  
data_2014=drop_columns(results_union,data_2014) 

In [195]:
#frames = [data_2012, data_2013, data_2014]

data_payments_wa = pd.concat([data_2012, data_2013, data_2014])


In [197]:
data_payments_wa.columns

Index([u'npi', u'last_org_name', u'first_name', u'mi', u'credentials',
       u'gender', u'street1', u'street2', u'city', u'zip', u'state',
       u'country', u'type', u'place_of_service', u'hcpcs_code',
       u'hcpcs_description', u'hcpcs_drug_indicator', u'line_srvc_cnt',
       u'bene_unique_cnt', u'bene_day_srvc_cnt', u'allowed_amt',
       u'submitted_chrg_amt', u'payment_amt', u'report_year', u'zip_base'],
      dtype='object')

In [194]:
.shape

(524609, 25)

In [199]:
#save database
import cPickle as pickle
data_payments_wa.to_pickle('wa_payments_pickle.pkl')  

In [200]:
#load database
#data_payments_wa= pd.read_pickle('wa_payments_pickle.pkl')

In [201]:
data_payments_wa_pickle.shape

(524609, 25)

In [202]:
data_payments_wa_pickle.columns

Index([u'npi', u'last_org_name', u'first_name', u'mi', u'credentials',
       u'gender', u'street1', u'street2', u'city', u'zip', u'state',
       u'country', u'type', u'place_of_service', u'hcpcs_code',
       u'hcpcs_description', u'hcpcs_drug_indicator', u'line_srvc_cnt',
       u'bene_unique_cnt', u'bene_day_srvc_cnt', u'allowed_amt',
       u'submitted_chrg_amt', u'payment_amt', u'report_year', u'zip_base'],
      dtype='object')

In [1]:
#data_2013.head(20).T

In [2]:
#data_provider_wa.info()

In [ ]:
#keep=['npi','nppes_provider_last_org_name', 'nppes_provider_first_name','nppes_provider_zip']

In [ ]:
#data_all=data_all[keep]

In [ ]:
#data_all.head(3)

In [ ]:
#data_all.info()

In [ ]:
#data_all.count()

In [ ]:
#data_all.dropna().count()

In [ ]:
#data_all=data_all.dropna()

In [ ]:
#data_all.head(3)

In [ ]:
#data_all.info()

In [ ]:
#data_all['nppes_provider_zip']=data_all['nppes_provider_zip'].astype(str)

In [ ]:
#data_all.head(3)

In [ ]:
#data_all['nppes_provider_zip']=data_all['nppes_provider_zip'].str.slice(0,5)

In [ ]:
#data_all.head(15)

In [ ]:
#data_all.info()

In [ ]:
#data_all['nppes_provider_zip']=data_all['nppes_provider_zip'].astype(int)

In [ ]:
#data_all

In [ ]:
#def add_names(x, y):
#    return x + ' ' + y

In [ ]:
#data_all.groupby('nppes_provider_zip').count()

In [ ]:
#data_all.to_csv('providers_names_and_zips.csv')

In [ ]:
# def find_names(a):
#     df = pd.read_csv('7_move_javascript/providers_names_and_zips.csv')
#     for zipcode in df['nppes_provider_zip']:
#         if zipcode == a:
#             firs_nm = df['nppes_provider_first_name'].values()
#             last_nm = df['nppes_provider_last_org_name'].values()
#             break
#     return firs_nm, last_nm


In [ ]:
#df2=df1[df1['nppes_provider_zip']==98101]

In [ ]:
#df2['nppes_provider_last_org_name'].values[0]

In [ ]:
#last_nm = df2['nppes_provider_last_org_name'].values[0]
#first_nm = df2['nppes_provider_first_name'].values[0]

In [ ]:
#print last_nm

In [ ]:
#print first_nm